# Transfer Learning

Using the high level transfer learning APIs, you can easily customize pretrained models for feature extraction or fine-tuning. 

In this notebook, we will use a pre-trained Inception_V1 model. But we will operate on the pre-trained model to freeze first few layers, replace the classifier on the top, then fine tune the whole model. And we use the fine-tuned model to solve the dogs-vs-cats classification problem,

## Preparation

### 1. Get the dogs-vs-cats datasets

Download the training dataset from https://www.kaggle.com/c/dogs-vs-cats and extract it. 

The following commands copy about 1100 images of cats and dogs into demo/cats and demo/dogs separately. 
```shell
mkdir -p demo/dogs
mkdir -p demo/cats
cp train/cat.7* demo/cats
cp train/dog.7* demo/dogs```

### 2. Get the pre-trained Inception-V1 model

Download the pre-trained Inception-V1 model from [Zoo](https://s3-ap-southeast-1.amazonaws.com/bigdl-models/imageclassification/imagenet/bigdl_inception-v1_imagenet_0.4.0.model) 
 Alternatively, user may also download pre-trained caffe/Tensorflow/keras model.

In [ ]:
sc.stop()
spark.stop()

In [ ]:
%pylab inline

In [1]:
import re

from bigdl.nn.criterion import CrossEntropyCriterion
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType

from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Dense, Input, Flatten
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *
import numpy as np
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
from glob import glob



Adding /usr/lib64/python2.7/site-packages/bigdl/share/lib/bigdl-0.6.0-jar-with-dependencies.jar to BIGDL_JARS
Prepending /usr/lib64/python2.7/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
Adding /usr/lib/python2.7/site-packages/zoo/share/lib/analytics-zoo-bigdl_0.6.0-spark_2.1.0-0.2.0-jar-with-dependencies.jar to BIGDL_JARS
Prepending /usr/lib/python2.7/site-packages/zoo/share/conf/spark-analytics-zoo.conf to sys.path


/usr/lib64/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:169: RuntimeWarning: numpy

creating: createDefault
creating: createSGD
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor


In [2]:
from bigdl.nn.criterion import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType, ArrayType
from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Input, Flatten, Dense
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *
from zoo.feature.image.imagePreprocessing import *

import random
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from zoo.pipeline.api.keras.metrics import AUC
from pyspark.sql.types import *

import numpy as np
from sklearn.metrics import roc_auc_score
from pyspark.ml import Pipeline
from bigdl.optim.optimizer import * 

/usr/lib64/python2.7/site-packages/sklearn/utils/__init__.py:9: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/usr/lib64/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/usr/lib64/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/usr/lib64/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/usr/lib64/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _el

sparkConf = SparkConf().setAppName("testMutipleLabels")
sc = init_nncontext(sparkConf)

In [3]:
sparkConf = create_spark_conf().setAppName("testMutipleLabels")
sc = init_nncontext(sparkConf)
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [4]:
path="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/red_shirt/"
print path
new_path="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/"
print new_path

/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/red_shirt/
/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/


In [5]:
type(sc)

pyspark.context.SparkContext

In [6]:
from bigdl.util.common import *
from bigdl.transform.vision.image import *
import matplotlib.pyplot as plt

In [7]:
model_path = "/home/mahmood/analytics-zoo/demo/bigdl_inception-v1_imagenet_0.4.0.model"


In [8]:
image_path='/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/'
label_path = '/home/mahmood/analytics-zoo/apps/dogs-vs-cats/AllLabels.csv'

In [9]:
getLabel = udf(lambda x: text_to_label(x), DoubleType())

In [10]:
import pandas as pd
labelDF=pd.read_csv(label_path)

/usr/lib64/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/lib64/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/lib64/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/lib64/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/lib64/python2.7/site-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility

In [11]:
labelDF.head()

,Image_index,labels
0,black_jeans_00000000.jpg,black_jeans
1,black_jeans_00000001.jpeg,black_jeans
2,black_jeans_00000002.jpeg,black_jeans
3,black_jeans_00000003.jpg,black_jeans
4,black_jeans_00000004.jpg,black_jeans


In [12]:
sqlCtx = SQLContext(sc)
labelDFsp = sqlCtx.createDataFrame(labelDF)

In [13]:
label_texts = list("""black,jeans,blue,dress,shirt,red""".replace("\n", "").split(","))
print label_texts

label_map = {k: v for v, k in enumerate(label_texts)}
print label_map

['black', 'jeans', 'blue', 'dress', 'shirt', 'red']
{'blue': 2, 'shirt': 4, 'jeans': 1, 'black': 0, 'dress': 3, 'red': 5}


In [14]:
def text_to_label(text):
    arr = [0.0] * len(label_texts)
    for l in text.split("_"):
        arr[label_map[l]] = 1.0
     
    return arr
print text_to_label("black_jeans")   

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [15]:
getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))

In [16]:
labelDF11 = labelDFsp.select("Image_index", "labels") \
   .withColumn("label", getLabel(col('labels')))



In [17]:
labelDF11.select("Image_index","label").show()

+--------------------+--------------------+
|         Image_index|               label|
+--------------------+--------------------+
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0,

In [18]:
#getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))
getName = udf(lambda row: os.path.basename(row[0]), StringType())
imageDF = NNImageReader.readImages(image_path, sc, resizeH=300, resizeW=300 ) \
    .withColumn("Image_index", getName(col('image')))


labelDF = spark.read.load(label_path, format="csv", sep=",", inferSchema="true", header="true") \
   .select("Image Index", "Finding Labels") \
   .withColumn("label", getLabel(col('Finding Labels')))

In [ ]:
imageDF.head(1)

# MY code start from here

In [19]:
train_df = imageDF.join(labelDF11, on="Image_index", how="inner")
(trainingDF, validationDF) = train_df.randomSplit([0.7, 0.3])

## Fine-tune a pre-trained model

We fine-tune a pre-trained model by removing the last few layers, freezing the first few layers, and adding some new layers.

In [20]:
transformer = ChainedPreprocessing(
        [RowToImageFeature(), ImageResize(256, 256), ImageCenterCrop(224, 224),
         ImageChannelNormalize(123.0, 117.0, 104.0), ImageMatToTensor(), ImageFeatureToTensor()])

creating: createRowToImageFeature
creating: createImageResize
creating: createImageCenterCrop
creating: createImageChannelNormalize
creating: createImageMatToTensor
creating: createImageFeatureToTensor
creating: createChainedPreprocessing


### Load a pre-trained model

We use the Net API to load a pre-trained model, including models saved by Analytics Zoo, BigDL, Torch, Caffe and Tensorflow. Please refer to [Net API Guide](https://analytics-zoo.github.io/master/#APIGuide/PipelineAPI/net/).

In [21]:
full_model = Net.load_bigdl(model_path)

### Remove the last few layers

Here we print all the model layers and you can choose which layer(s) to remove.

When a model is loaded using Net, we can use the newGraph(output) api to define a Model with the output specified by the parameter. 

In [22]:
for layer in full_model.layers:
    print (layer.name())
model = full_model.new_graph(["pool5/drop_7x7_s1"])

data
conv1/7x7_s2
conv1/relu_7x7
pool1/3x3_s2
pool1/norm1
conv2/3x3_reduce
conv2/relu_3x3_reduce
conv2/3x3
conv2/relu_3x3
conv2/norm2
pool2/3x3_s2
inception_3a/3x3_reduce
inception_3a/5x5_reduce
inception_3a/relu_3x3_reduce
inception_3a/relu_5x5_reduce
inception_3a/pool
inception_3a/1x1
inception_3a/3x3
inception_3a/5x5
inception_3a/pool_proj
inception_3a/relu_pool_proj
inception_3a/relu_5x5
inception_3a/relu_3x3
inception_3a/relu_1x1
inception_3a/output
inception_3b/3x3_reduce
inception_3b/5x5_reduce
inception_3b/relu_3x3_reduce
inception_3b/relu_5x5_reduce
inception_3b/pool
inception_3b/1x1
inception_3b/3x3
inception_3b/5x5
inception_3b/pool_proj
inception_3b/relu_pool_proj
inception_3b/relu_5x5
inception_3b/relu_3x3
inception_3b/relu_1x1
inception_3b/output
pool3/3x3_s2
inception_4a/3x3_reduce
inception_4a/5x5_reduce
inception_4a/relu_3x3_reduce
inception_4a/relu_5x5_reduce
inception_4a/pool
inception_4a/1x1
inception_4a/3x3
inception_4a/5x5
inception_4a/pool_proj
inception_4a/relu_

The returning model's output layer is "pool5/drop_7x7_s1".

### Freeze some layers

We freeze layers from input to pool4/3x3_s2 inclusive.

model.freeze_up_to(["pool4/3x3_s2"])

In [23]:
inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(6, activation="sigmoid")(flatten)
lrModel = Model(inputNode, logits)

creating: createZooKerasInput
creating: createZooKerasFlatten
creating: createZooKerasDense
creating: createZooKerasModel


### Add a few new layers

In [24]:
val_summary = ValidationSummary(log_dir="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/logDirectroy/log", app_name="testMutipleLabels")

creating: createValidationSummary


In [25]:
classifier = NNEstimator(lrModel, MultiLabelSoftMarginCriterion(), transformer, SeqToTensor([6])) \
     .setLearningRate(0.001).setBatchSize(24).setMaxEpoch(10).setFeaturesCol("image")\
     .setCachingSample(False)\
     .setValidation(EveryEpoch(), validationDF,[AUC()],  24)
#\
 #    .setValidationSummary(val_summary)
#\
    # .setCheckpoint("/home/mahmood/analytics-zoo/apps/dogs-vs-cats/logDirectroy/checkpoint", EveryEpoch(),False)

creating: createMultiLabelSoftMarginCriterion
creating: createSeqToTensor
creating: createFeatureLabelPreprocessing
creating: createNNEstimator
creating: createEveryEpoch
creating: createAUC


In [26]:
nnModel = classifier.fit(trainingDF)
print("Finished training")

creating: createToTuple
creating: createChainedPreprocessing
Finished training


In [27]:
nnModel.transform(trainingDF).show(5)

predictionDF = nnModel.transform(validationDF).cache()

predictionDF.select("Image_index","label","prediction").show(5)

+--------------------+--------------------+-----------+--------------------+--------------------+
|         Image_index|               image|     labels|               label|          prediction|
+--------------------+--------------------+-----------+--------------------+--------------------+
|black_jeans_00000...|[file:/home/mahmo...|black_jeans|[1.0, 1.0, 0.0, 0...|[0.0024215402, 0....|
|black_jeans_00000...|[file:/home/mahmo...|black_jeans|[1.0, 1.0, 0.0, 0...|[6.489417E-4, 0.1...|
|blue_dress_000002...|[file:/home/mahmo...| blue_dress|[0.0, 0.0, 1.0, 1...|[0.0055481303, 0....|
|blue_jeans_000001...|[file:/home/mahmo...| blue_jeans|[0.0, 1.0, 1.0, 0...|[0.0072195423, 0....|
|blue_jeans_000001...|[file:/home/mahmo...| blue_jeans|[0.0, 1.0, 1.0, 0...|[0.013826376, 0.8...|
+--------------------+--------------------+-----------+--------------------+--------------------+
only showing top 5 rows

+--------------------+--------------------+--------------------+
|         Image_index|      

In [ ]:
#---------------------AUC  CALCULATION  ----------------------------

from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import seaborn as sns; sns.set_style('whitegrid')

#------------------------------------------------------------------------
Lab=predictionDF.select("label") #.collect()
Pre=predictionDF.select("prediction") #.collect()


#__________________________________________







In [ ]:
type(Lab)



In [ ]:
P=np.array(Lab)

In [ ]:
type(P)

In [ ]:
P

In [ ]:
P=np.array(Lab)
P=P.reshape(P.shape[0],P.shape[2])
Pre1=np.array(Pre)
Pre1=Pre1.reshape(Pre1.shape[0],Pre1.shape[2])

In [ ]:
total_score=roc_auc_score(P, Pre1)
n_classes=6

#----------------[ Get AUC values ]-------------------------------------
def get_auc_values(LabelArray,PredArray): 
    n_classes=6
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(LabelArray[:, i], PredArray[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    return roc_auc,fpr,tpr     

roc_auc,fpr, tpr=get_auc_values(P,Pre1)


In [ ]:
#---------------------------------plot AUC---------------------------------
def ploting_AUC(fpr, tpr, label_texts): 

  #%matplotlib inline
    plt.figure()
    lw=1
    colors = (['aqua', 'darkorange', 'cornflowerblue','red','blue','maroon'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                label='{0} (area = {1:0.2f})'
                ''.format(label_texts[i], roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--', lw=lw)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('AUC for multi-class')
      
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
            fancybox=True, shadow=True, ncol=5)
    plt.show()

ploting_AUC(fpr, tpr, label_texts)